In [1]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

pd.set_option('display.mpl_style', 'default') # Make the graphs a bit prettier
plt.rcParams['figure.figsize'] = (10, 7)

In [6]:
data = pd.read_csv('/home/jvdzwaan/Downloads/1999_CHES_dataset_means.csv')
print data.shape
data[data['country']==10]

(142, 50)


,eastwest,eumember,country,year,expert,party_id,party,vote,family,govt,...,fin,fr,ger,gr,irl,it,nl,por,sw,uk
96,1,1,10,1999,11,1001,CDA,18.400000,4,0,...,0,0,0,0,0,0,1,0,0,0
97,1,1,10,1999,11,1002,PvdA,29.000000,5,1,...,0,0,0,0,0,0,1,0,0,0
98,1,1,10,1999,11,1003,VVD,24.699999,3,1,...,0,0,0,0,0,0,1,0,0,0
99,1,1,10,1999,11,1004,D'66,9.000000,3,1,...,0,0,0,0,0,0,1,0,0,0
100,1,1,10,1999,11,1005,GL,7.300000,7,0,...,0,0,0,0,0,0,1,0,0,0
101,1,1,10,1999,11,1006,SGP,1.800000,10,0,...,0,0,0,0,0,0,1,0,0,0
102,1,1,10,1999,11,1007,GPV,1.300000,10,0,...,0,0,0,0,0,0,1,0,0,0
103,1,1,10,1999,11,1008,RPF,2.000000,10,0,...,0,0,0,0,0,0,1,0,0,0
104,1,1,10,1999,11,1009,CD,0.000000,1,0,...,0,0,0,0,0,0,1,0,0,0
105,1,1,10,1999,11,1014,SP,3.500000,6,0,...,0,0,0,0,0,0,1,0,0,0
